In [1]:
import copy
from enum import Enum
DEBUG=False
class NFA_Enum(Enum):
    START=0
    TERMINAL=1
    EPSILON=2
    FAILURE=3
    INTERNAL=4
class NFA:
    START=NFA_Enum.START
    TERMINAL=NFA_Enum.TERMINAL
    EPSILON=NFA_Enum.EPSILON
    FAILURE=NFA_Enum.FAILURE
    INTERNAL=NFA_Enum.INTERNAL
    def __init__(self,state=START,transitions=None):
        self.transitions=transitions if transitions is not None else {}
        self.state=state
    def copy(self):
        return NFA(self.state,copy.deepcopy(self.transitions))
    def __call__(self,transition=None):
        if transition in self.transitions:
            if NFA.EPSILON in self.transitions[transition].transitions:
                return self.transitions[transition](NFA.EPSILON)
            return self.transitions[transition]
        if self.state==NFA.TERMINAL:
            return self
        if NFA.EPSILON in self.transitions:
            return self.transitions[NFA.EPSILON](transition)
        return NFA(NFA.FAILURE)
    def add_edge(self,transition,node=None):
        if transition in self.transitions:
                raise ValueError(f"Transition {transition}->{node} already exists")
        if node is None:
            node=NFA(NFA.TERMINAL)
        self.transitions[transition]=node
        if node.state == NFA.TERMINAL:
            node.state=NFA.INTERNAL
        return self
    def add_edges(self,other: 'NFA'):
        for transition,node in other.transitions.items():
            self.add_edge(transition,node)
    def add_self_loop(self, transition):
        self.transitions[transition] = self
        return self
    def __repr__(self) -> str:
        return f"{self.state=},{self.transitions=}"
    @classmethod
    def from_expr(cls,expr,convert_unit=lambda x:x):
        if DEBUG:
            print('from_expr',cls,expr,convert_unit)
        return cls.parse_expr(expr,convert_unit)[0]
    @classmethod
    def parse_expr(cls,expr,convert_unit):#converts expr to a nfa returns the start and end nodes

        #| for OR
        #? for optional
        #() for grouping
        # for concatenation
        #equal precedence for OR and concatenation
        #words for units - regex [a-zA-Z]+
        #DOES NOT WORK for expressions like (x y)|(x? z) instead do (x y|z)|z
        #set convert_unit when working with NFA_funcs and convert names to functions
        escape=False
        stack=[cls(cls.START)]
        operator = ' '
        i=0
        az="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
        last_optional=False
        while i<len(expr):
            sub_start,sub_end=None,None
            match expr[i],escape:    
                case '\\',False:
                    escape=True
                case '(',False:#could optimize by saving location of internal pairs
                    open_paren=1
                    sub_expr_len=0
                    while open_paren>0:
                        i+=1
                        sub_expr_len+=1
                        match expr[i],escape:
                            case '(',False:
                                open_paren+=1
                            case ')',False:
                                open_paren-=1
                            case '\\',False:
                                escape=True
                            case _:
                                escape=False#escape is consumed regardless of the character following
                    sub_start,sub_end=cls.parse_expr(expr[i-sub_expr_len+1:i],convert_unit)         
                case s,False if s in " |":
                    operator=s
                case '?',False:
                    last_optional=True
                case _:
                    unit=""
                    while i<len(expr):
                        match expr[i],escape:
                            case '\\',False:
                                escape=True
                            case s,False if s in r"()| ":
                                break
                            case _:
                                unit+=expr[i]
                                escape=False
                        i+=1
                    sub_end=cls(NFA.INTERNAL)
                    sub_start=cls(NFA.INTERNAL,transitions={convert_unit(unit):sub_end})
                    i-=1
                    escape=False

            if sub_start is not None:
                if operator == ' ':
                    stack[-1].add_edges(sub_start)#instead of an epsilon transition
                    stack.append(sub_end)
                    if last_optional:
                        stack[-3].add_edges(sub_start)
                        last_optional=False
                elif operator == '|':
                    stack[-2].add_edges(sub_start)
                    sub_end.add_edge(NFA.EPSILON,stack[-1])
                    #treat x?|y as (x|y)?
            i+=1
        stack[-1].state=NFA.TERMINAL
        if last_optional:
            stack[-2].state=NFA.TERMINAL
        return stack[0],stack[-1]
class NFA_funcs(NFA):
    #the more general the case the later it should be in the list
    #NFA_funcs.from_expr(r"lambda\ x:x>5")
    EPSILON_FUNCTION=lambda unit:NFA.EPSILON#equivalent to using NFA.EPSILON when using __call__
    def __call__(self,unit):
        for transition,node in self.transitions.items():
            if transition(unit):
                if NFA.EPSILON in node.transitions:
                    return self.transitions[NFA.EPSILON]
                return node
        if self.state==NFA.TERMINAL:
            return self
        if NFA.EPSILON in self.transitions:
            return self.transitions[NFA.EPSILON](transition)
        return NFA(NFA.FAILURE)
    @classmethod
    def from_expr(cls,expr,convert_unit=eval):#by default will try to evaluate names as functions
        return super().parse_expr(expr,convert_unit)[0]
    #NFA_funcs(NFA.START,{bool: NFA_funcs(NFA.INTERNAL,{lambda x:x>5:NFA(NFA.TERMINAL)})})(True)(6)
class NFA_funcs_converters:
    @staticmethod
    def type_check(t:str):
        return lambda x: isinstance(x,eval(t))
    @staticmethod
    def in_check(collection):
        return lambda x: x in collection#or just use collection.__contains__ if the .__contains__ method is defined and does not change
    @staticmethod
    def equal_check(val):
        return lambda x: x==val   

In [3]:
import pyautogui as pag
from time import sleep
from string import ascii_uppercase
"""shift quote comma shift quote period J O I N shift nine left bracket F shift
quote shift backslash shift left bracket I shift plus shift right bracket shift backslash shift quote"""
function_numbers = {
    'one':'1',
    'two':'2',
    'three':'3',
    'four':'4',
    'five':'5',
    'six':'6',
    'seven':'7',
    'eight':'8',
    'nine':'9',
    'ten':'10',
    'eleven':'11',
    'twelve':'12',
}
# Define patterns for each character
patterns = {
  '`': 'backtick|(grave accent)',
 '[': 'opening (square)? bracket',
 ']': 'closing (square)? bracket',
 '\\': 'backslash',
 ';': 'semicolon',
 "'": '(single? quote)|apostrophe',
 ',': 'comma',
 ' ': 'space',
 '.': 'period|dot',
 '/': 'forward slash',
 '=': 'equals|equal sign',
 '-': 'dash|minus|hyphen',
 }|{
   '~':'tilde',
    '!':'exclamation mark',
    '@':'at sign',
    '#':'hashtag|pound sign',
    '$':'dollar sign',
    '%':'percent sign',
    '^':'caret',
    '&':'ampersand',
    '*':'asterisk',
    '(':'opening parenthesis',
    ')':'closing parenthesis',
    '_':'underscore',
    '+':'plus',
    '{':'opening (curly bracket)|brace',
    '}':'closing (curly bracket)|brace',
    '|':'pipe|(vertical bar)',
    ':':'colon',
    '"': 'double quote',
     '<': 'opening angle bracket',
   '>': 'closing angle bracket',
     '<': 'opening angle bracket',
 '>': 'closing angle bracket',
 }|{
   'shift':'shift',
    'ctrl':'control',
    'alt':'alt',
 }
pattern_nfas = {
    key:NFA.from_expr(pattern) for key,pattern in patterns.items()
}

state_commands = ['hold','release','function','repeat']
holds = ['shift','ctrl','alt' ]
uppers=list(ascii_uppercase)
other_string = ['enter','escape','backspace','delete','del','up','down','left','right']
def convert(unit,state={"nfas":pattern_nfas}):
  #patterns try to match first
  next_nfas={}
  for key,nfa in state["nfas"].items():
    x=nfa(unit)
    if  x.state == NFA.TERMINAL:
      #reset nfas
      state["nfas"]=pattern_nfas
      return key
    elif x.state != NFA.FAILURE:
      next_nfas[key]=nfa(unit)
    
  state["nfas"]=next_nfas if next_nfas else pattern_nfas#reset when all have failed
  if unit in uppers+state_commands+other_string:#pyautogui supports these other keyboard inputs directly
    return unit.lower()
  if unit in function_numbers:
    return function_numbers[unit]
  return None#invalid unit or patterns incomplete

commands={#for those that do not use pyautogui.hotkey for output
          #clicks will reset the queue
  "click":pag.click,
  "right click":pag.rightClick,
}

def output_transcription_unit(unit,state={'hold':False,'release':False,'function':False,'repeat':False,'last_pressed':None,'queued':[]}):
  if DEBUG:
    print('output_transcription_unit',unit,state)
  unit=convert(unit)
  if unit is None:
    return None
  elif unit in state_commands:
    state[unit]=True
  elif unit in commands:
    commands[unit]()
  elif state['hold']:
    pag.keyDown(unit)
    print("key down",unit)  
    state['release']=False
    state['hold']=False
    if unit in state:
        state[unit]=True
  elif state['release']:
    pag.keyUp(unit)
    print("key up",unit)  
    state['release']=False
    state['hold']=False
    if unit in state['queued']:
        queued=[x for x in queued if x!=unit]
    if unit in state:
        state[unit]=False
  elif unit in holds:
      state['queued'].append(unit)
  elif unit in function_numbers.values() and state['function']:
      state['function']=False
      output_transcription_unit('f'+unit)
  elif state['repeat']:
    ... 
  else:
    state['last_pressed']=[*state['queued'],unit]
    pag.hotkey(*state['last_pressed'])
    print("press", "+".join(state['last_pressed']))
    state['queued']=[] 

#sample = "control shift S"#"""P R I N T opening parenthesis shift quote shift H E L L O shift W O R L D left left left left left space"""
#for word in sample.split(): 
#  output_transcription_unit(word)

In [4]:
class Converter:
    def __init__(self,nfas=pattern_nfas):
        self.nfas_orig = nfas
        self.reset()
    def reset(self):
        self.nfas={k:v for k,v in self.nfas_orig.items()}#don't need to deepcopy the nfas
    def __call__(self, unit):
        for key, nfa in self.nfas.items():
            x = nfa(unit)
            if x.state == NFA.TERMINAL:
                self.reset()
                return key
            elif x.state != NFA.FAILURE:
                self.nfas[key] = nfa(unit)
        if not self.nfas:
            self.reset()
        if unit in uppers + state_commands + other_string:
            return unit.lower()
        if unit in function_numbers:
            return function_numbers[unit]
        return None

converter = Converter()

In [5]:
from typing import Any
from functools import partial
class Commmand:
    def __init__(self,function,nfa,ignore_None=False):
        self.nfa_start=nfa
        self.function=function
        self.ignore_None=ignore_None
        self.reset()
    @classmethod
    def _constructed_call(cls,function,nfa,ignore_None):
        return cls(function,nfa,ignore_None).__call__
    @classmethod
    def dec(cls,nfa,ignore_None=False):
        return partial(cls._constructed_call,nfa=nfa,ignore_None=ignore_None)
    def reset(self):
        self.nfa=self.nfa_start
        self.args=[]
        self.reset_next_call=False
class PrefixCommand(Commmand):
    def __call__(self, unit):
        if self.reset_next_call:
            self.reset()
        if unit is None and self.ignore_None:
            return self
        self.args.append(unit)
        self.nfa = self.nfa(unit)
        if self.nfa.state == NFA.TERMINAL:
            self.reset_next_call=True
            return self.function(*self.args)
        elif self.nfa.state == NFA.FAILURE:
            self.reset_next_call=True
            return NFA(NFA.FAILURE)
        #else self.nfa.state == NFA.INTERNAL
        return self
class PostfixCommand(Commmand):
    #identical to Prefix command but with reversed args and nfa
    def __call__(self, unit):
        if self.reset_next_call:
            self.reset()
        if unit is None and self.ignore_None:
            return self
        self.args.append(unit)
        self.nfa = self.nfa(unit)
        if self.nfa.state == NFA.TERMINAL:
            self.reset_next_call=True
            return self.function(*self.args[::-1])
        elif self.nfa.state == NFA.FAILURE:
            self.reset_next_call=True
            return NFA.FAILURE
        #else self.nfa.state == NFA.INTERNAL
        return self# for chaining
    def call_on_stack(self,stack):
        #moves from back to front by repeatedly calling the command if the command is unfinished after exhausting the stack returns failure
        i=-1
        while i>=-len(stack):
            x=self(stack[i])
            if x == NFA.FAILURE:
                return NFA.FAILURE
            if x!=self:
                return x
            i-=1
        return NFA.FAILURE
class InfixCommand(PostfixCommand):
    def __init__(self, function, pre_nfa,post_nfa, ignore_None_pre=False,ignore_None_post=False,differentiate_args=False):
        self.function=function
        if not differentiate_args:
            self.function = lambda pre,post:function(*pre,*post)
        self.pre_nfa_start=pre_nfa#ugly but works
        self.post_nfa_start=post_nfa
        self.ignore_None_pre=ignore_None_pre
        self.ignore_None_pre
        self.reset()
    def reset(self):
        self.pre_nfa=self.pre_nfa_start
        self.post_nfa=self.post_nfa_start
        self.pre_args=[]
        self.post_args=[]
        self.mode="pre"
        self.reset_next_call=False
    def __call__(self,unit):
        if self.reset_next_call:
            self.reset()
        if self.mode == "pre":
            if unit is None and self.ignore_None_pre:
                return self
            self.pre_args.append(unit)
            self.pre_nfa = self.pre_nfa(unit)
            if self.pre_nfa.state == NFA.TERMINAL:
                self.mode="post"
            elif self.pre_nfa.state == NFA.FAILURE:
                self.reset_next_call=True
                return NFA.FAILURE
        else:
            if unit is None and self.ignore_None_post:
                return self
            self.post_args.append(unit)
            self.post_nfa = self.post_nfa(unit)
            if self.post_nfa.state == NFA.TERMINAL:
                self.reset_next_call=True
                return self.function(self.pre_args,self.post_args)
            elif self.post_nfa.state == NFA.FAILURE:
                self.reset_next_call=True
                return NFA.FAILURE
        return self
state_commands = ['hold','release','function','repeat']
holds = ['shift','ctrl','alt' ]
uppers=list(ascii_uppercase)
other_string = ['enter','escape','backspace','delete','del','up','down','left','right']
#TODO
def lang_to_int(s:str):
    raise NotImplementedError
#TODO
from PIL import Image
class OutputTranscriber:
    def hold(self,  unit):
        pag.keyDown(unit)
        if DEBUG:
            print("key down",unit)  
    def release(self,unit):
        pag.keyUp(unit)
        if DEBUG: 
            print("key up",unit)
        if unit in self.queued:
                self.queued=[x for x in self.queued if x!=unit]
    def repeat(self,unit,amount):#repeats last unit NOT last action
        amount=lang_to_int(amount)#need to convert natural language to int
        for _ in len(range(amount)):
            self(unit)
    def fn(self,unit):
        self(f'f{unit}')
    def click(self):#Don't need to indicate as command since it takesa no args
        pag.click()
    def right_click(self):
        pag.rightClick()
    def move_to_img(self,img_name):
        #need some handling for getting image name
        raise NotImplementedError
        #zoom factor is part of the vscode state and needs to be inputted before
        image=Image(img_name)
        image = image.resize((int(image.width  * zoom_factor), int(image.height * zoom_factor)))
        # Locate the image on the screen
        try:
            image_location = pag.locateOnScreen(image,confidence=.75,grayscale=True)
            #so that icons that have numbers appear over them show up
            x, y = pag.center(image_location)
            pag.moveTo(x, y)
        except pag.ImageNotFoundException:
            print("Image not found")        
    def __init__(self):
        self.hold = PrefixCommand(self.hold,NFA.from_expr(r"shift|ctrl|alt"))#could support others 
        self.release = PrefixCommand(self.release,NFA.from_expr(r"shift|ctrl|alt"))
        self.queued = []
        prefix_commands = {'hold':self.hold,'release':self.release}
o=OutputTranscriber()

In [ ]:
from functools import wraps
def dbg(func):
    @wraps(func)
    def new(*args,**kwargs):
        print(func.__name__,args,kwargs)
        return func(*args,**kwargs)
    return new